# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [131]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [132]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [133]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [134]:
data1 = data.groupby(['CustomerID','ProductName']).agg({"Quantity":sum})
data1

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [135]:
data2 = data1.pivot_table(index = 'ProductName', columns = 'CustomerID', values="Quantity", fill_value = 0)
data2

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [106]:
from scipy.spatial.distance import pdist, squareform
distances = pdist(data2.T, 'euclidean')
print(squareform(distances))

[[   0.           11.91637529   10.48808848 ...,  228.62851966  239.
   229.77380181]
 [  11.91637529    0.           11.74734012 ...,  228.01096465
   239.03765394  229.70415756]
 [  10.48808848   11.74734012    0.         ...,  228.08112592
   238.26665734  229.77380181]
 ..., 
 [ 228.62851966  228.01096465  228.08112592 ...,    0.          304.13812651
   305.16389039]
 [ 239.          239.03765394  238.26665734 ...,  304.13812651    0.
   303.10889132]
 [ 229.77380181  229.70415756  229.77380181 ...,  305.16389039
   303.10889132    0.        ]]


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [107]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data2.T, 'euclidean'))), 
                         index=data2.columns, columns=data2.columns)

distances



CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


In [145]:
similarities = distances[33].sort_values(ascending=False)[1:6]
similarities1 = list(similarities.index)
similarities1

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [146]:
data3 = data1.loc[similarities1]
data3

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [110]:
data4 = data3.groupby(['ProductName']).agg({"Quantity":sum})

data4_sorted = data4.sort_values(by=['Quantity'], ascending=False)
data4_sorted


,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [122]:
data5 = data4_sorted.merge(data2[33], on = ['ProductName'])
data5

,Quantity,33
ProductName,,
Butter - Unsalted,3,0
Wine - Ej Gallo Sierra Valley,3,0
Towels - Paper / Kraft,3,1
Soup - Campbells Bean Medley,3,0
Wine - Blue Nun Qualitatswein,3,0
...,...,...
Hinge W Undercut,1,0
Ice Cream Bar - Hageen Daz To,1,0
Jagermeister,1,0


In [125]:
data6 = data5.loc[data5[33] == 0]
data6.head()

,Quantity,33
ProductName,,
Butter - Unsalted,3,0
Wine - Ej Gallo Sierra Valley,3,0
Soup - Campbells Bean Medley,3,0
Wine - Blue Nun Qualitatswein,3,0
Chicken - Soup Base,2,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [142]:
dicti = {}

lista_customers = list(set(data2.columns))

lista_customers

[83973,
 49159,
 22536,
 18441,
 59399,
 92168,
 86028,
 75791,
 96272,
 32785,
 47122,
 75793,
 51224,
 40985,
 28702,
 33,
 6185,
 18480,
 86065,
 94258,
 67636,
 81981,
 90174,
 91194,
 10311,
 28751,
 51279,
 90193,
 30803,
 28756,
 41045,
 55380,
 55381,
 12381,
 30815,
 86111,
 16481,
 67682,
 88159,
 92255,
 65650,
 57459,
 18552,
 86136,
 8316,
 79997,
 77950,
 82057,
 2187,
 65678,
 86167,
 57497,
 18591,
 61601,
 71844,
 4261,
 14501,
 20648,
 90284,
 6317,
 8367,
 22705,
 10418,
 32946,
 8372,
 88245,
 41147,
 90300,
 18622,
 97324,
 28867,
 65731,
 39110,
 47303,
 200,
 47305,
 28875,
 6352,
 45266,
 35028,
 94420,
 35034,
 65757,
 6367,
 22751,
 78048,
 26850,
 82146,
 16614,
 78054,
 94438,
 30954,
 59626,
 90347,
 12529,
 82161,
 24820,
 18680,
 65785,
 18683,
 51451,
 53501,
 63742,
 88318,
 18688,
 45313,
 78082,
 92418,
 43270,
 57606,
 264,
 16650,
 96522,
 71948,
 96524,
 12561,
 18706,
 69906,
 16664,
 2329,
 78104,
 33051,
 51483,
 55580,
 63771,
 18720,
 35107,
 

In [164]:
for i in lista_customers:
    
    similarities = distances[i].sort_values(ascending=False)[1:6]
    similarities1 = list(similarities.index)
    data = data1.loc[similarities1]
    data = data.groupby(['ProductName']).agg({"Quantity":sum}).reset_index()
    data_sorted = data.sort_values(by=['Quantity'], ascending=False)
    data = data_sorted.merge(data2[i], on = ['ProductName'])
    data = data.loc[data[i] == 0]
    data = data[1:6]
    data = list(data.ProductName)
    dicti[i] = data
    
dicti


{'CustomerID': 'ProductName',
 83973: ['Veal - Slab Bacon',
  'Bandage - Fexible 1x3',
  'Ezy Change Mophandle',
  'Chicken - Wieners',
  'Puree - Passion Fruit'],
 49159: ['Cookies - Assorted',
  'Puree - Mocha',
  'Cheese - Parmesan Grated',
  'Cheese - Victor Et Berthold',
  'Chicken - Leg, Boneless'],
 22536: ['Cheese - Mozzarella',
  'Cheese Cloth No 100',
  'Bread - Italian Roll With Herbs',
  'Bread - Hot Dog Buns',
  'Table Cloth - 53x69 Colour'],
 18441: ['Anchovy Paste - 56 G Tube',
  'Sprouts - Baby Pea Tendrils',
  'Lemonade - Natural, 591 Ml',
  'Sausage - Breakfast',
  'Soup Knorr Chili With Beans'],
 59399: ['Knife Plastic - White',
  'Towels - Paper / Kraft',
  'Nantucket - Pomegranate Pear',
  'Snapple - Iced Tea Peach',
  'Cookie Chocolate Chip With'],
 92168: ['Veal - Brisket, Provimi,bnls',
  'Snapple Lemon Tea',
  'Spinach - Baby',
  'Pop Shoppe Cream Soda',
  'Bread - Calabrese Baguette'],
 86028: ['Pork - Hock And Feet Attached',
  'Vinegar - Tarragon',
  'Bread 

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [166]:
final = pd.DataFrame(dicti)
final

,CustomerID,83973,49159,22536,18441,59399,92168,86028,75791,96272,...,45042,55281,59379,75762,77810,16375,77814,20476,67582,65535
0,ProductName,Veal - Slab Bacon,Cookies - Assorted,Cheese - Mozzarella,Anchovy Paste - 56 G Tube,Knife Plastic - White,"Veal - Brisket, Provimi,bnls",Pork - Hock And Feet Attached,Mussels - Cultivated,Bread Crumbs - Panko,...,Spinach - Baby,"Pepper - Paprika, Hungarian",Thermometer Digital,Soupcontfoam16oz 116con,Pants Custom Dry Clean,"Beef - Chuck, Boneless",Cake - Cake Sheet Macaroon,"Pasta - Detalini, White, Fresh",Ecolab - Solid Fusion,Scallop - St. Jaques
1,ProductName,Bandage - Fexible 1x3,Puree - Mocha,Cheese Cloth No 100,Sprouts - Baby Pea Tendrils,Towels - Paper / Kraft,Snapple Lemon Tea,Vinegar - Tarragon,Vaccum Bag 10x13,Guinea Fowl,...,Ice Cream Bar - Oreo Cone,Soupcontfoam16oz 116con,"Pork - Back, Short Cut, Boneless",Vol Au Vents,Sword Pick Asst,"Wine - Red, Colio Cabernet",Sea Bass - Whole,"Cheese - Boursin, Garlic / Herbs",Wine - Redchard Merritt,"Chestnuts - Whole,canned"
2,ProductName,Ezy Change Mophandle,Cheese - Parmesan Grated,Bread - Italian Roll With Herbs,"Lemonade - Natural, 591 Ml",Nantucket - Pomegranate Pear,Spinach - Baby,Bread Foccacia Whole,Turkey - Oven Roast Breast,Sunflower Seed Raw,...,Beets - Mini Golden,Brandy - Bar,Beans - Wax,Ketchup - Tomato,Arizona - Green Tea,Rum - Mount Gay Eclipes,Squid U5 - Thailand,Beans - Kidney White,Mussels - Frozen,"Bread - Roll, Soft White Round"
3,ProductName,Chicken - Wieners,Cheese - Victor Et Berthold,Bread - Hot Dog Buns,Sausage - Breakfast,Snapple - Iced Tea Peach,Pop Shoppe Cream Soda,Lettuce - Spring Mix,Bread - Calabrese Baguette,Bread - English Muffin,...,Wine - Redchard Merritt,Vinegar - Tarragon,Sponge Cake Mix - Chocolate,Anchovy Paste - 56 G Tube,Pail For Lid 1537,Sauce - Hollandaise,Guinea Fowl,Wine - Blue Nun Qualitatswein,Cheese - Bocconcini,Chicken - Wieners
4,ProductName,Puree - Passion Fruit,"Chicken - Leg, Boneless",Table Cloth - 53x69 Colour,Soup Knorr Chili With Beans,Cookie Chocolate Chip With,Bread - Calabrese Baguette,Thermometer Digital,Grouper - Fresh,Sprouts - Alfalfa,...,Beef - Montreal Smoked Brisket,"Bread - Roll, Soft White Round",Pastry - Butterscotch Baked,Cheese - Taleggio D.o.p.,"Seedlings - Mix, Organic","Nut - Chestnuts, Whole",Macaroons - Two Bite Choc,Lime Cordial - Roses,"Peas - Pigeon, Dry","Lamb - Pieces, Diced"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [167]:
distances = pdist(data2.T, 'jaccard')
print(squareform(distances))

[[ 0.          0.94308943  0.88785047 ...,  1.          1.          1.        ]
 [ 0.94308943  0.          0.92741935 ...,  1.          1.          1.        ]
 [ 0.88785047  0.92741935  0.         ...,  1.          1.          1.        ]
 ..., 
 [ 1.          1.          1.         ...,  0.          0.90551181
   0.95081967]
 [ 1.          1.          1.         ...,  0.90551181  0.          0.9       ]
 [ 1.          1.          1.         ...,  0.95081967  0.9         0.        ]]


In [168]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data2.T, 'jaccard'))), 
                         index=data2.columns, columns=data2.columns)

distances


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.514644,0.529703,0.524038,0.513043,0.516129,0.510549,0.509174,0.511211,0.513889,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
200,0.514644,1.000000,0.518828,0.514523,0.503704,0.512000,0.520325,0.510204,0.518828,0.519313,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
264,0.529703,0.518828,1.000000,0.515695,0.527027,0.515419,0.510460,0.508772,0.515419,0.511013,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
356,0.524038,0.514523,0.515695,1.000000,0.517094,0.510917,0.508403,0.521531,0.510823,0.513393,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
412,0.513043,0.503704,0.527027,0.517094,1.000000,0.512605,0.516529,0.520362,0.510373,0.515284,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.518987,0.525581,0.515021,0.521951,0.520362,1.000000,0.524887,0.525346,0.511628,0.504000
98069,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.516129,0.512821,0.514644,0.511013,0.526549,0.524887,1.000000,0.527027,0.517510,0.523404
98159,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.512500,0.525822,0.520000,0.513889,0.515152,0.525346,0.527027,1.000000,0.524793,0.512605


In [169]:
dicti = {}

lista_customers = list(set(data2.columns))

for i in lista_customers:
    
    similarities = distances[i].sort_values(ascending=False)[1:6]
    similarities1 = list(similarities.index)
    data = data1.loc[similarities1]
    data = data.groupby(['ProductName']).agg({"Quantity":sum}).reset_index()
    data_sorted = data.sort_values(by=['Quantity'], ascending=False)
    data = data_sorted.merge(data2[i], on = ['ProductName'])
    data = data.loc[data[i] == 0]
    data = data[1:6]
    data = list(data.ProductName)
    dicti[i] = data
    
dicti

{83973: ['Milk - 1%',
  'Chocolate - Semi Sweet, Calets',
  'Pork - Belly Fresh',
  'Bread Crumbs - Japanese Style',
  'Bread - Rye'],
 49159: ['Cookie Dough - Double',
  'Mussels - Cultivated',
  'Cheese - Bocconcini',
  'Macaroons - Two Bite Choc',
  'Soup - Campbells Bean Medley'],
 22536: ['Cocktail Napkin Blue',
  'Puree - Passion Fruit',
  'Isomalt',
  'Nut - Pistachio, Shelled',
  'Chinese Foods - Chicken'],
 18441: ['Macaroons - Two Bite Choc',
  'Bread - English Muffin',
  'Juice - Cranberry, 341 Ml',
  'Beef - Ground, Extra Lean, Fresh',
  'Nantucket - Pomegranate Pear'],
 59399: ['Cheese - Brie, Triple Creme',
  'Rum - Coconut, Malibu',
  'Cheese - Parmesan Cubes',
  'Gloves - Goldtouch Disposable',
  'Wine - Hardys Bankside Shiraz'],
 92168: ['Wiberg Super Cure',
  'Salmon - Sockeye Raw',
  'Cumin - Whole',
  'Scallops - 10/20',
  'Tea - English Breakfast'],
 86028: ['Lettuce - Treviso',
  'Bread - Calabrese Baguette',
  'Appetizer - Mini Egg Roll, Shrimp',
  'Ice Cream Bar

In [170]:
final = pd.DataFrame(dicti)
final

,83973,49159,22536,18441,59399,92168,86028,75791,96272,32785,...,45042,55281,59379,75762,77810,16375,77814,20476,67582,65535
0,Milk - 1%,Cookie Dough - Double,Cocktail Napkin Blue,Macaroons - Two Bite Choc,"Cheese - Brie, Triple Creme",Wiberg Super Cure,Lettuce - Treviso,Beer - Labatt Blue,Vinegar - Sherry,"Placemat - Scallop, White",...,Dc - Frozen Momji,Wine - White Cab Sauv.on,Broom - Corn,Beer - Blue,Veal - Inside,Ice Cream Bar - Oreo Cone,Cumin - Whole,Broom - Corn,"Lamb - Pieces, Diced",Tea - English Breakfast
1,"Chocolate - Semi Sweet, Calets",Mussels - Cultivated,Puree - Passion Fruit,Bread - English Muffin,"Rum - Coconut, Malibu",Salmon - Sockeye Raw,Bread - Calabrese Baguette,Tia Maria,Zucchini - Yellow,Soup - Campbells Tomato Ravioli,...,Bread Fig And Almond,"Mushroom - Trumpet, Dry",Coffee - Dark Roast,Beef - Short Loin,Knife Plastic - White,Muffin Batt - Blueberry Passion,Beef - Montreal Smoked Brisket,Mustard - Seed,Pie Filling - Cherry,Wine - Chablis 2003 Champs
2,Pork - Belly Fresh,Cheese - Bocconcini,Isomalt,"Juice - Cranberry, 341 Ml",Cheese - Parmesan Cubes,Cumin - Whole,"Appetizer - Mini Egg Roll, Shrimp",Sea Bass - Whole,Langers - Ruby Red Grapfruit,Tomato - Tricolor Cherry,...,Garbag Bags - Black,Bread - Rye,Chocolate - Compound Coating,Ketchup - Tomato,Island Oasis - Mango Daiquiri,"Ice - Clear, 300 Lb For Carving",Soupcontfoam16oz 116con,"Cheese - Brie,danish","Beer - Alexander Kieths, Pale Ale",Berry Brulee
3,Bread Crumbs - Japanese Style,Macaroons - Two Bite Choc,"Nut - Pistachio, Shelled","Beef - Ground, Extra Lean, Fresh",Gloves - Goldtouch Disposable,Scallops - 10/20,Ice Cream Bar - Oreo Cone,Bread - English Muffin,Cookie Dough - Double,Chips Potato Salt Vinegar 43g,...,Grapes - Red,Truffle Cups - Brown,Muffin Batt - Choc Chk,Bread - Calabrese Baguette,Wine - Alsace Gewurztraminer,Kellogs All Bran Bars,Squid - Tubes / Tenticles 10/20,Mayonnaise - Individual Pkg,Rambutan,"Sole - Dover, Whole, Fresh"
4,Bread - Rye,Soup - Campbells Bean Medley,Chinese Foods - Chicken,Nantucket - Pomegranate Pear,Wine - Hardys Bankside Shiraz,Tea - English Breakfast,Coffee - Dark Roast,"Fish - Scallops, Cold Smoked",Tea - Jasmin Green,Sausage - Breakfast,...,Scallops - 10/20,Lettuce - Frisee,Mayonnaise - Individual Pkg,Pasta - Cheese / Spinach Bauletti,Bread - English Muffin,Clam Nectar,"Lemonade - Natural, 591 Ml",Chef Hat 20cm,Sponge Cake Mix - Chocolate,Soupcontfoam16oz 116con
